In [1]:
%pip install --upgrade protobuf
%pip install -q supervision
%pip install -q ultralytics
%pip install -q streamlit
%pip install -q stqdm
%pip install -q streamlit-image-select
%pip install -q ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.2 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -q ngrok-stable-linux-amd64.zip

In [3]:
!./ngrok authtoken 2WfKUvHcOZJ5V6Ma3NHiGA3C2xc_28pt3zUCjntbnmKShbb7h

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
def get_link():
  get_ipython().system_raw('./ngrok http 8501 &')
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
  'import sys, json; print("Ссылка для streamlit: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

In [5]:
utils = '''
import numpy as np
import supervision as sv
from glob import glob

def get_file_paths(folder_with_labels):
    return glob(folder_with_labels, recursive=True)

def is_iopa(ground_truth, pred, conf=.95):

    # Координаты области пересечения.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])

    # Высота и ширина пересечения.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))

    # Площадь пересечения
    area_of_intersection = i_height * i_width

    # Высота и ширина Предсказания
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1

    # Площадь Прогноза
    pred_area = pd_height*pd_width

    # IoPA интерсекшин овер предикт ареа :)
    iopa = area_of_intersection/pred_area

    if iopa >= conf: return True
    else: return False

def mrg_detection(detections_seg, detections_det, conf=0.95)->sv.detection:

  xyxy_lst=[]
  mask_lst=[]
  confidence_lst=[]
  class_id_lst=[]
  tracker_id_lst=[]
  ret_lst=[]

  ret_detections = sv.Detections.empty()

  for s in range(len(detections_seg.xyxy)):
    for d in range(len(detections_det.xyxy)):
      iopa = is_iopa(detections_seg.xyxy[s],detections_det.xyxy[d], conf)
      if iopa:

        if detections_seg.xyxy is not None:
          xyxy_lst.append(detections_seg.xyxy[s])
        else:
          xyxy_lst = None

        if detections_seg.mask is not None:
          mask_lst.append(detections_seg.mask[s])
        else:
          mask_lst = None

        if detections_seg.confidence is not None:
          confidence_lst.append(detections_seg.confidence[s])
        else:
          confidence_lst = None

        if detections_seg.class_id is not None:
          class_id_lst.append(detections_seg.class_id[s])
        else:
          class_id_lst = None

        if detections_seg.tracker_id is not None:
          tracker_id_lst.append(detections_seg.tracker_id[s])
        else:
          tracker_id_lst = None

  if detections_seg.xyxy is not None:
    ret_detections.xyxy=np.asarray(xyxy_lst, dtype=np.float32)
  else:
    ret_detections.xyxy=None

  if detections_seg.mask is not None:
    ret_detections.mask=np.asarray(mask_lst)
  else:
    ret_detections.mask=None

  if detections_seg.confidence is not None:
    ret_detections.confidence=np.asarray(confidence_lst, dtype=np.float32)
  else:
   ret_detections.confidenc=None

  if detections_seg.class_id is not None:
    ret_detections.class_id=np.asarray(class_id_lst, dtype=np.int64)
  else:
    ret_detections.class_id=None

  if detections_seg.tracker_id is not None:
    ret_detections.tracker_id=np.asarray(tracker_id_lst, dtype=np.int64)
  else:
    ret_detections.tracker_id=None

  return ret_detections
  '''

with open('utils.py', 'w') as f:
    f.write(utils)


In [6]:
from google.colab import drive

# Подключаем гугл диск для загрузки файлов
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [7]:
# Переносим наши веса
!cp /content/drive/MyDrive/Стажировка/weights/best-3_classes-m-a100.pt /content/best-3_classes-m-a100.pt
!cp /content/drive/MyDrive/Стажировка/weights/yolov8m-seg.pt /content/yolov8m-seg.pt

In [47]:
code = '''
import streamlit as st
import supervision as sv
from stqdm import stqdm
import numpy as np
import ultralytics as ul
import ffmpeg
from utils import is_iopa, mrg_detection, get_file_paths
from pathlib import Path
from streamlit_image_select import image_select

vid_pth = "/content/videos"
img_pth = "/content/images"
Path(vid_pth).mkdir(parents=False, exist_ok=True)
Path(img_pth).mkdir(parents=False, exist_ok=True)

# Цвет Аннотатора
color = sv.Color.from_hex("ff0000")

# Цвет текста Аннотатора
text_color = sv.Color.from_hex("#0ff")

# Создаем Аннотатор
halo_annotator = sv.HaloAnnotator(color=color)
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_CENTER, color=color, text_color=text_color, text_padding=2, text_thickness=1)

with st.spinner('Загружаем модели...'):
  model_seg = ul.YOLO('yolov8m-seg.pt')
  model_det = ul.YOLO('best-3_classes-m-a100.pt')


uploaded_file = st.file_uploader("Выберите файл для загрузки и обработки",type = ['mp4'])

if uploaded_file is not None:

    bytes_data = uploaded_file.getvalue()

    with open(uploaded_file.name, 'wb') as f:
      f.write(bytes_data)

    video_info = sv.VideoInfo.from_video_path(uploaded_file.name)

    #if st.checkbox('Просмотреть загруженное видео'):
    st.write('Res:', video_info.resolution_wh, 'Frames:', video_info.total_frames, 'FPS:', video_info.fps, 'Dur.:', video_info.total_frames//video_info.fps, 'sec.')
    st.video(uploaded_file.name)

    with st.sidebar:
      inter = st.slider('Определите интервал в фреймах', 1, video_info.total_frames, (video_info.total_frames//3, video_info.total_frames//3*2))
      st.write('~Значения в секундах:', 'c', inter[0]//video_info.fps, 'по', inter[1]//video_info.fps)

      nfr = st.slider('Определите шаг', 1, video_info.fps, video_info.fps//2)
      st.write("Шаг:", nfr, 'кадров')

      cnf = st.slider('Конфиденс', 0.1, 0.9, 0.5, step=0.05)

      b1 = st.button("Применить", type="primary")

    if b1:

      # Инициализируем полигон
      polygon = np.array([
          [10, 10],
          [10, video_info.height-10],
          [video_info.width-10, video_info.height-10],
          [video_info.width-10, 10]
      ])

      # Создаем зону
      zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

      # Создаем аннотатор зоны
      zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=5, text_thickness=2, text_scale=1)

      # Создаем генератор фреймов
      frames_generator = sv.get_video_frames_generator(uploaded_file.name, stride=nfr, start=inter[0], end=inter[1])

      # Создаем итератор по генератору
      frame_iterator = iter(frames_generator)

      with sv.VideoSink(target_path = vid_pth + '/a_' + uploaded_file.name, video_info=video_info, codec='mp4v') as vsink:
        with sv.ImageSink(target_dir_path = img_pth, overwrite=True) as isink:
          zcount=0
          for frame in stqdm(frame_iterator, total=inter[1]-inter[0], desc="Детекция и аннотирование фреймов...", st_container=st.sidebar):

              # Детектим персону
              result_seg = model_seg(frame, imgsz=video_info.width, verbose=False, conf=cnf)[0]
              detections_seg = sv.Detections.from_ultralytics(result_seg)
              detections_seg = detections_seg[(detections_seg.class_id == 0)]

              # Детектим каску
              result_det = model_det(frame, imgsz=video_info.width, verbose=False, conf=cnf)[0]
              detections_det = sv.Detections.from_ultralytics(result_det)
              detections_det = detections_det[(detections_det.class_id == 1) & (detections_det.confidence > cnf)]
              labels = [f"{confidence:.0%}" for _, _, confidence, _, _ in detections_det]

              # Для детекции обновляем трекер и тригер зоны
              trg = zone.trigger(detections=detections_det)

              # Объединяем модели с коэффициентом погружения bb головы в bb персоны 0.89
              detections_sd = mrg_detection(detections_seg, detections_det, conf=0.89)

              # Аннотируем фрейм
              if len(detections_sd)>0:
                frame= halo_annotator.annotate(scene=frame.copy(), detections=detections_sd)
                frame = label_annotator.annotate(scene=frame.copy(), detections=detections_det, labels=labels)

              frame = zone_annotator.annotate(scene=frame.copy())

              if (len(detections_sd)>0) & (sum(trg)>zcount):
                  isink.save_image(image=frame)

              zcount = sum(trg)

              vsink.write_frame(frame=frame)

      #if st.checkbox('Просмотреть итоговое видео'):
      with st.spinner('Подготавливаем видео для просмотра...'):
          # Конвертируем видео в кодек H.264
          #ffmpeg.input(vid_pth+'/a_'+uploaded_file.name).output(vid_pth+'/b_'+uploaded_file.name, vcodec="libx264").run(overwrite_output=True)
          ffmpeg.input(vid_pth+'/a_'+uploaded_file.name).output(vid_pth+'/b_'+uploaded_file.name, vcodec="libx264").run(overwrite_output=True)
      st.video(vid_pth + '/b_' + uploaded_file.name)

      # Получаем список путей ко всем файлам картинок с их именами
      img_file_list = get_file_paths(img_pth + '/**' + '/*.png')

      img = image_select("Выбор фрейма", img_file_list, use_container_width=False)
      st.image(img)

st.snow()
'''

with open('dwn_bin_file.py', 'w') as f:
    f.write(code)



In [49]:
get_link()

Ссылка для streamlit: https://c409-34-80-200-42.ngrok-free.app


In [50]:
!streamlit run dwn_bin_file.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.200.42:8501

WARNING ⚠️ NMS time limit 0.550s exceeded

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --e